In [1]:
import csv
import tensorflow as tf
import numpy as np
import urllib

In [2]:
# DO NOT CHANGE THIS CODE
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[1:]))
    return ds.batch(batch_size).prefetch(1)


def solution_A5():
    data_url = 'https://github.com/dicodingacademy/assets/raw/main/Simulation/machine_learning/sunspots.csv'
    urllib.request.urlretrieve(data_url, 'sunspots.csv')

    time_step = []
    sunspots = []

    with open('sunspots.csv') as csvfile:
      reader = csv.reader(csvfile, delimiter=',')
      next(reader)
      for row in reader:
        sunspots.append(float(row[2]))
        time_step.append(int(row[0]))

    series= np.array(sunspots)

    # Normalization Function. DO NOT CHANGE THIS CODE
    min=np.min(series)
    max=np.max(series)
    series -= min
    series /= max
    time=np.array(time_step)

    # DO NOT CHANGE THIS CODE
    split_time=3000


    time_train=  time[:split_time]
    x_train = series[:split_time]
    time_valid = time[split_time:]
    x_valid = series[split_time:]

    # DO NOT CHANGE THIS CODE
    window_size=30
    batch_size=32
    shuffle_buffer_size=1000


    train_set=windowed_dataset(x_train, window_size=window_size,
                               batch_size=batch_size, shuffle_buffer=shuffle_buffer_size)


    model=tf.keras.models.Sequential([
      tf.keras.layers.Conv1D(filters=64, kernel_size=5,
                               strides=1, padding="causal",
                               activation="relu",
                               input_shape=[None, 1]),
        tf.keras.layers.LSTM(128, return_sequences=True),
        tf.keras.layers.LSTM(128, return_sequences=True),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(32, activation="relu"),
        tf.keras.layers.Dense(10, activation="relu"),
        tf.keras.layers.Dense(1)
    ])

    model.compile(loss=tf.keras.losses.Huber(),
                  optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
                  metrics=["mae"])

    model.fit(train_set, epochs=10)
    
    return model

In [3]:
# The code below is to save your model as a .h5 file.
# It will be saved automatically in your Submission folder.
if __name__ == '__main__':
    # DO NOT CHANGE THIS CODE
    model=solution_A5()
    model.save("model_A5.h5")


Epoch 1/10
93/93 [==============================] - 6s 36ms/step - loss: 0.0149 - mae: 0.1232
Epoch 2/10
93/93 [==============================] - 3s 31ms/step - loss: 0.0066 - mae: 0.0808
Epoch 3/10
93/93 [==============================] - 3s 32ms/step - loss: 0.0047 - mae: 0.0669
Epoch 4/10
93/93 [==============================] - 3s 32ms/step - loss: 0.0036 - mae: 0.0596
Epoch 5/10
93/93 [==============================] - 3s 33ms/step - loss: 0.0031 - mae: 0.0551
Epoch 6/10
93/93 [==============================] - 3s 33ms/step - loss: 0.0028 - mae: 0.0527
Epoch 7/10
93/93 [==============================] - 3s 34ms/step - loss: 0.0026 - mae: 0.0513
Epoch 8/10
93/93 [==============================] - 3s 33ms/step - loss: 0.0025 - mae: 0.0502
Epoch 9/10
93/93 [==============================] - 3s 32ms/step - loss: 0.0024 - mae: 0.0495
Epoch 10/10
93/93 [==============================] - 3s 32ms/step - loss: 0.0023 - mae: 0.0489


c:\Users\ilham\miniconda3\envs\vision1\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
